This notebook is a work in progress. I will clean up code, add comments, etc when I get a chance.

# Multiclass Synthetic Data Experimentation

All testing done in this notebook is for proof of concept.
<br>
The following sections experiment with synthetic generation for multiclass classification. Furthermore, since Linear Discriminant Analysis runs much faster than Random Forest, the SMOTE classifiers are tested on LD instead of RF classifiers

In [1]:
import numpy as np
import pandas as pd
import imblearn
import sklearn
import smote_variants
import os.path
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import smote_variants as sv
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

print(f"numpy version {np.__version__}")
print(f"pandas version {pd.__version__}")
print(f"matplotlib version {matplotlib.__version__}")
print(f"seaborn version {sns.__version__}")
print(f"imblearn version {imblearn.__version__}")
print(f"sklearn version {sklearn.__version__}")
print(f"smote_variants version {smote_variants.__version__}")

numpy version 1.18.1
pandas version 1.0.1
matplotlib version 3.1.3
seaborn version 0.11.1
imblearn version 0.8.1
sklearn version 0.24.1
smote_variants version 0.4.0


In [21]:
# Read csv files into a dataframe
df = pd.read_csv("../../../Data/cleaned/Darknet_reduced_features.csv")

# SMOTE doesn't work with one hot encoding (at least from what I could find), so applications are mapped to a number
traffic_application_to_num = {
    'audio-streaming': 0,
    'browsing': 1,
    'chat': 2,
    'email': 3,
    'file-transfer': 4,
    'p2p': 5,
    'video-streaming': 6,
    'voip': 7
}

# Same as above but for the traffic type
traffic_type_to_num = {
    'Non-Tor': 0,
    'NonVPN': 1,
    'Tor': 2,
    'VPN': 3
}

# Map the categorical features to numbers
df['Application Type'] = df['Application Type'].map(traffic_application_to_num)
df['Traffic Type'] = df['Traffic Type'].map(traffic_type_to_num)

# Specify the columns as ints. Otherwise, SMOTE will create values like 3.2.
df['Application Type'] = df['Application Type'].astype(int)
df['Traffic Type'] = df['Traffic Type'].astype(int)

In [22]:
# Create x and y dataframes
X = df.loc[:, df.columns != 'Application Type']
# Decide if we should comment out next line or not
X = X.drop(['Traffic Type'], axis = 1)
y = df['Application Type']

# Baseline Linear Discriminant Analysis

In [11]:
# Perform a baseline linear discriminant analysis test so that we can compare to a model trained on the SMOTE data
ld_classifier = LinearDiscriminantAnalysis()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
ld_classifier.fit(X_train, y_train)

y_pred = ld_classifier.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.6134572569530339
F1: 0.5977860142716336


In [14]:
# Print a confusion matrix
print(metrics.classification_report(y_test, y_pred, target_names=['audio-streaming','browsing','chat','email','file-transfer',
                                                          'p2p','video-streaming','voip']))

                 precision    recall  f1-score   support

audio-streaming       0.61      0.64      0.62      3581
       browsing       0.65      0.70      0.68      6654
           chat       0.10      0.08      0.09      2272
          email       0.31      0.56      0.40      1246
  file-transfer       0.61      0.24      0.35      2255
            p2p       0.77      0.85      0.81      9656
video-streaming       0.42      0.36      0.39      1928
           voip       0.45      0.09      0.15       705

       accuracy                           0.61     28297
      macro avg       0.49      0.44      0.44     28297
   weighted avg       0.61      0.61      0.60     28297



# Testing a single SMOTE variant with Linear Discriminant Analysis

In [24]:
# Convert the data and target datasets to numpy arrays
X = X.to_numpy()
y = y.to_numpy()

In [25]:
# The smote_variants package wants specific labels for the data
dataset= {'data': X, 'target': y, 'name': 'darknet_multiclass'}

In [26]:
# This can be stored anywhere; however, it takes a lot of space, so I'm using an external drive. 
# We will need to change the location later
cache_path= "/mnt/sda1"

if not os.path.exists(cache_path):
    os.makedirs(cache_path)

# Custom attempt to try and use many variations with different proportions
The library does not natively allow for multiple variations and proportions on multiclass data, so I'm trying to improvise. The code below is bad and needs fixed. I was just trying to see if it was possible.

In [82]:
ranges = [0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
samplers = ["sv.OUPS(","sv.SMOTE_D(","sv.NT_SMOTE(","sv.ROSE(","sv.NDO_sampling(","sv.SMOTE("]

In [88]:
sampler_variations = []
for ra in ranges:
    for sampler in samplers:
        sampler_variations.append(f"{sampler}proportion={ra})")

In [89]:
print(sampler_variations)

['sv.OUPS(proportion=0.25)', 'sv.SMOTE_D(proportion=0.25)', 'sv.NT_SMOTE(proportion=0.25)', 'sv.ROSE(proportion=0.25)', 'sv.NDO_sampling(proportion=0.25)', 'sv.SMOTE(proportion=0.25)', 'sv.OUPS(proportion=0.5)', 'sv.SMOTE_D(proportion=0.5)', 'sv.NT_SMOTE(proportion=0.5)', 'sv.ROSE(proportion=0.5)', 'sv.NDO_sampling(proportion=0.5)', 'sv.SMOTE(proportion=0.5)', 'sv.OUPS(proportion=0.75)', 'sv.SMOTE_D(proportion=0.75)', 'sv.NT_SMOTE(proportion=0.75)', 'sv.ROSE(proportion=0.75)', 'sv.NDO_sampling(proportion=0.75)', 'sv.SMOTE(proportion=0.75)', 'sv.OUPS(proportion=1)', 'sv.SMOTE_D(proportion=1)', 'sv.NT_SMOTE(proportion=1)', 'sv.ROSE(proportion=1)', 'sv.NDO_sampling(proportion=1)', 'sv.SMOTE(proportion=1)', 'sv.OUPS(proportion=1.25)', 'sv.SMOTE_D(proportion=1.25)', 'sv.NT_SMOTE(proportion=1.25)', 'sv.ROSE(proportion=1.25)', 'sv.NDO_sampling(proportion=1.25)', 'sv.SMOTE(proportion=1.25)', 'sv.OUPS(proportion=1.5)', 'sv.SMOTE_D(proportion=1.5)', 'sv.NT_SMOTE(proportion=1.5)', 'sv.ROSE(propor

In [91]:
best_f1 = 0
best_acc = 0

for sampler in sampler_variations: 
    print(sampler)
    oversampler = sv.MulticlassOversampling(eval(sampler))
    X_samp, y_samp= oversampler.sample(X, y)
    X_samp_df = pd.DataFrame(X_samp, columns = ['Protocol', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
           'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
           'Fwd Packet Length Max', 'Fwd Packet Length Min',
           'Fwd Packet Length Mean', 'Fwd Packet Length Std',
           'Bwd Packet Length Max', 'Bwd Packet Length Min',
           'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
           'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
           'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
           'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
           'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
           'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
           'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max',
           'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
           'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count',
           'ACK Flag Count', 'Down/Up Ratio', 'Average Packet Size',
           'Fwd Segment Size Avg', 'Bwd Segment Size Avg', 'Bwd Packet/Bulk Avg',
           'Bwd Bulk Rate Avg', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
           'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
           'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Idle Mean', 'Idle Std',
           'Idle Max', 'Idle Min'])
    y_samp_df = pd.DataFrame(y_samp, columns = ['Application Type'])
    ld_classifier = LinearDiscriminantAnalysis()

    X_train, X_test, y_train, y_test = train_test_split(X_samp_df, y_samp_df, test_size=0.2)
    ld_classifier.fit(X_train, y_train)

    y_pred = ld_classifier.predict(X_test)

    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("F1:",metrics.f1_score(y_test, y_pred, average='weighted'))
    best_f1 = max(best_f1, metrics.f1_score(y_test, y_pred, average='weighted'))
    best_acc = max(best_acc, metrics.accuracy_score(y_test, y_pred))
print(best_acc, best_f1)

2022-02-22 20:25:47,582:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-02-22 20:25:47,596:INFO:MulticlassOversampling: Sampling minority class with label: 0
2022-02-22 20:25:47,608:INFO:OUPS: Running sampling via ('OUPS', "{'proportion': 1.0, 'n_jobs': 1, 'random_state': None}")


sv.OUPS(proportion=0.25)


2022-02-22 20:25:48,234:INFO:MulticlassOversampling: Sampling minority class with label: 1
2022-02-22 20:25:48,247:INFO:OUPS: Running sampling via ('OUPS', "{'proportion': 0.38574503101658436, 'n_jobs': 1, 'random_state': None}")
/home/drake/miniconda3/envs/gpu2/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
2022-02-22 20:25:48,832:INFO:MulticlassOversampling: Sampling minority class with label: 2
2022-02-22 20:25:48,847:INFO:OUPS: Running sampling via ('OUPS', "{'proportion': 0.27629076644268274, 'n_jobs': 1, 'random_state': None}")
/hom

Accuracy: 0.5346366340841479
F1: 0.5342940049972364
sv.SMOTE_D(proportion=0.25)


/home/drake/miniconda3/envs/gpu2/lib/python3.6/site-packages/smote_variants/_smote_variants.py:10454: RuntimeWarning: invalid value encountered in true_divide
  p_ij = dist[:, 1:]/np.sum(dist[:, 1:], axis=1)[:, None]
2022-02-22 20:26:13,252:INFO:MulticlassOversampling: Sampling minority class with label: 1
2022-02-22 20:26:13,260:INFO:SMOTE_D: Running sampling via ('SMOTE_D', "{'proportion': 0.38574503101658436, 'k': 3, 'n_jobs': 1, 'random_state': None}")
/home/drake/miniconda3/envs/gpu2/lib/python3.6/site-packages/smote_variants/_smote_variants.py:10454: RuntimeWarning: invalid value encountered in true_divide
  p_ij = dist[:, 1:]/np.sum(dist[:, 1:], axis=1)[:, None]
2022-02-22 20:26:17,716:INFO:MulticlassOversampling: Sampling minority class with label: 2
2022-02-22 20:26:17,732:INFO:SMOTE_D: Running sampling via ('SMOTE_D', "{'proportion': 0.27629076644268274, 'k': 3, 'n_jobs': 1, 'random_state': None}")
/home/drake/miniconda3/envs/gpu2/lib/python3.6/site-packages/smote_variants/_s

...

Accuracy: 0.4038798433635614
F1: 0.3864429784142381
sv.NDO_sampling(proportion=2)


2022-02-22 21:05:07,042:INFO:MulticlassOversampling: Sampling minority class with label: 1
2022-02-22 21:05:07,055:INFO:NDO_sampling: Running sampling via ('NDO_sampling', "{'proportion': 0.38574503101658436, 'n_neighbors': 5, 'T': 0.5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 21:05:39,792:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.38574503101658436, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': <module 'numpy.random' from '/home/drake/miniconda3/envs/gpu2/lib/python3.6/site-packages/numpy/random/__init__.py'>}")
2022-02-22 21:05:44,191:INFO:MulticlassOversampling: Sampling minority class with label: 2
2022-02-22 21:05:44,206:INFO:NDO_sampling: Running sampling via ('NDO_sampling', "{'proportion': 0.27629076644268274, 'n_neighbors': 5, 'T': 0.5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 21:06:09,981:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.27629076644268274, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': <module 'numpy.random' from '/

Accuracy: 0.47654992078111114
F1: 0.45019764737457063
sv.SMOTE(proportion=2)


2022-02-22 21:08:20,634:INFO:MulticlassOversampling: Sampling minority class with label: 1
2022-02-22 21:08:20,644:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.38574503101658436, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 21:08:24,881:INFO:MulticlassOversampling: Sampling minority class with label: 2
2022-02-22 21:08:24,897:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.27629076644268274, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 21:08:26,577:INFO:MulticlassOversampling: Sampling minority class with label: 4
2022-02-22 21:08:26,597:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.2041857337335367, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 21:08:28,302:INFO:MulticlassOversampling: Sampling minority class with label: 6
2022-02-22 21:08:28,325:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 0.1665091989761737, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
20

Accuracy: 0.46719136438582026
F1: 0.4392970460549596
0.6201481481481481 0.61525682320825


In [67]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from pipelinehelper import PipelineHelper

oversampler= sv.MulticlassOversampling(sv.distance_SMOTE())
classifier= LinearDiscriminantAnalysis()




param_grid= {'proportion': [0.25,0.5,0.75,1,1.5,2]}

# Specifying the gridsearch for model selection
grid= GridSearchCV(oversampler, param_grid= param_grid, cv= 2, n_jobs= 1, verbose= 2, scoring= 'accuracy')
grid.fit(X, y)


TypeError: estimator should be an estimator implementing 'fit' method, <smote_variants._smote_variants.MulticlassOversampling object at 0x7f90146c34a8> was passed

In [57]:
print(grid.best_score_)

0.5575308474649809


In [48]:
ld_classifier = LinearDiscriminantAnalysis()

# samp_obj and cl_obj contain the oversampling and classifier objects which give the
# best performance together
# Performs RepeatedStratifiedKFold with 5 splits and 3 repeats 
samp_obj, cl_obj= sv.model_selection(dataset= dataset,
                                        samplers= sv.get_n_quickest_oversamplers_multiclass(1),
                                        classifiers= [ld_classifier],
                                        cache_path= cache_path,
                                        score= 'brier',
                                        n_jobs= 5,
                                        max_samp_par_comb= 35)

2022-02-22 19:29:56,261:INFO:dataset: darknet_multiclass, samplings_available: True, evaluations_available: False
2022-02-22 19:29:56,262:INFO:doing the folding
2022-02-22 19:29:56,263:INFO:Folding reading from file folding_darknet_multiclass.pickle
2022-02-22 19:29:56,931:INFO:do the samplings
2022-02-22 19:29:56,932:INFO:create sampling objects, random_state: 
2022-02-22 19:29:56,932:INFO:samplers: [<class 'smote_variants._smote_variants.OUPS'>]
2022-02-22 19:29:56,933:INFO:[{'proportion': 0.1}, {'proportion': 0.25}, {'proportion': 0.5}, {'proportion': 0.75}, {'proportion': 1.0}, {'proportion': 1.5}, {'proportion': 2.0}]
2022-02-22 19:29:56,934:INFO:random_indices: [0 6 4 2 1 5 3]
2022-02-22 19:29:56,934:INFO:[{'proportion': 0.1}, {'proportion': 2.0}, {'proportion': 1.0}, {'proportion': 0.5}, {'proportion': 0.25}, {'proportion': 1.5}, {'proportion': 0.75}]
2022-02-22 19:29:56,935:INFO:{'proportion': 0.1, 'random_state': None}
2022-02-22 19:29:56,935:INFO:Sampling sampler parameter st

["('darknet_multiclass', OrderedDict([('proportion', 2.0), ('random_state', None)]), 'LinearDiscriminantAnalysis', OrderedDict([('covariance_estimator', None), ('n_components', None), ('priors', None), ('shrinkage', None), ('solver', 'svd'), ('store_covariance', False), ('tol', 0.0001)]))"]
["('darknet_multiclass', OrderedDict([('proportion', 1.5), ('random_state', None)]), 'LinearDiscriminantAnalysis', OrderedDict([('covariance_estimator', None), ('n_components', None), ('priors', None), ('shrinkage', None), ('solver', 'svd'), ('store_covariance', False), ('tol', 0.0001)]))"]
["('darknet_multiclass', OrderedDict([('proportion', 1.0), ('random_state', None)]), 'LinearDiscriminantAnalysis', OrderedDict([('covariance_estimator', None), ('n_components', None), ('priors', None), ('shrinkage', None), ('solver', 'svd'), ('store_covariance', False), ('tol', 0.0001)]))"]
["('darknet_multiclass', OrderedDict([('proportion', 0.75), ('random_state', None)]), 'LinearDiscriminantAnalysis', OrderedD

ValueError: multi_class must be in ('ovo', 'ovr')

In [27]:
# Testing out a random SMOTE variant
oversampler = sv.MulticlassOversampling(sv.distance_SMOTE())

X_samp, y_samp= oversampler.sample(X, y)

2022-02-22 19:14:31,914:INFO:MulticlassOversampling: Running multiclass oversampling with strategy eq_1_vs_many_successive
2022-02-22 19:14:31,924:INFO:MulticlassOversampling: Sampling minority class with label: 1
2022-02-22 19:14:31,937:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 19:14:47,840:INFO:MulticlassOversampling: Sampling minority class with label: 0
2022-02-22 19:14:47,853:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 0.38574503101658436, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 19:14:52,920:INFO:MulticlassOversampling: Sampling minority class with label: 2
2022-02-22 19:14:52,935:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 0.27629076644268274, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")
2022-02-22 19:14:55,431:INFO:MulticlassOversampling: Sampling minority class with label: 

In [28]:
# SMOTE gives us a numpy array, but dataframes are easier for humans to interpret
# Convert the numpy array to a dataframe with appropriate column labels
X_samp_df = pd.DataFrame(X_samp, columns = ['Protocol', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count',
       'ACK Flag Count', 'Down/Up Ratio', 'Average Packet Size',
       'Fwd Segment Size Avg', 'Bwd Segment Size Avg', 'Bwd Packet/Bulk Avg',
       'Bwd Bulk Rate Avg', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Idle Mean', 'Idle Std',
       'Idle Max', 'Idle Min'])
y_samp_df = pd.DataFrame(y_samp, columns = ['Application Type'])

In [144]:
# We went from 141,482 samples to 373,235 samples
#X_samp_df['Label']=X_samp_df['Label'].round()
#y_samp_df['Label1']=y_samp_df['Label1'].round()
#y_samp_df['Label1'] = y_samp_df['Label1'].astype(int)
#X_samp_df['Label'] = X_samp_df['Label'].astype(int)

In [37]:
y_samp_df['Application Type'] = y_samp_df.astype(int)

In [44]:
y_samp_df.groupby('Application Type').size()

Application Type
0    48520
1    48520
2    48520
3    48520
4    48520
5    48520
6    48520
7    48520
dtype: int64

In [39]:
ld_classifier = LinearDiscriminantAnalysis()

X_train, X_test, y_train, y_test = train_test_split(X_samp_df, y_samp_df, test_size=0.2)
ld_classifier.fit(X_train, y_train)

y_pred = ld_classifier.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred, average='weighted'))

/home/drake/miniconda3/envs/gpu2/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Accuracy: 0.47238252267106345
F1: 0.444528298374608


In [41]:
# Print a confusion matrix
print(metrics.classification_report(y_test, y_pred, target_names=['audio-streaming','browsing','chat','email','file-transfer',
                                                          'p2p','video-streaming','voip']))

                 precision    recall  f1-score   support

audio-streaming       0.47      0.58      0.52      9635
       browsing       0.50      0.67      0.57      9572
           chat       0.15      0.10      0.12      9798
          email       0.46      0.21      0.29      9747
  file-transfer       0.56      0.29      0.38      9823
            p2p       0.60      0.81      0.69      9697
video-streaming       0.50      0.38      0.43      9695
           voip       0.44      0.75      0.56      9665

       accuracy                           0.47     77632
      macro avg       0.46      0.47      0.45     77632
   weighted avg       0.46      0.47      0.44     77632



In [148]:
# Once again, to make the data easier for humans to interpret, convert the numbers to names
traffic_application_to_val = {
    0:'audio-streaming',
    1:'browsing',
    2:'chat',
    3:'email',
    4:'file-transfer',
    5:'p2p',
    6:'video-streaming',
    7:'voip'
}

traffic_type_to_val = {
    0:'Non-Tor',
    1:'NonVPN',
    2:'Tor',
    3:'VPN'
}

X_samp_df['Label'] = X_samp_df['Label'].map(traffic_type_to_val)
y_samp_df['Label1'] = y_samp_df['Label1'].map(traffic_application_to_val)

In [149]:
new_dataset = X_samp_df.join(y_samp_df)

In [150]:
new_dataset.to_csv("../../../Data/synthetic/SMOTE/test.csv")

In [121]:
df

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label1
0,6,229,1,1,0,0,0,0,0.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0
1,6,407,1,1,0,0,0,0,0.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0
2,6,431,1,1,0,0,0,0,0.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0
3,6,359,1,1,0,0,0,0,0.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0
4,6,10778451,591,400,64530,6659,131,0,109.187817,22.283313,...,0,0,0,0,1.437760e+15,3.117718e+06,1.437760e+15,1.437760e+15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141476,17,411806,2,0,44,0,22,22,22.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3,3
141477,17,411574,2,0,44,0,22,22,22.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3,3
141478,17,422299,2,0,44,0,22,22,22.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3,3
141479,17,411855,2,0,44,0,22,22,22.000000,0.000000,...,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3,3


In [122]:
new_dataset

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label1
0,6.0,2.290000e+02,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,audio-streaming,Non-Tor
1,6.0,4.070000e+02,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,audio-streaming,Non-Tor
2,6.0,4.310000e+02,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,audio-streaming,Non-Tor
3,6.0,3.590000e+02,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,audio-streaming,Non-Tor
4,6.0,1.077845e+07,591.000000,400.000000,6.453000e+04,6.659000e+03,131.000000,0.000000,109.187817,22.283313,...,0.0,0.0,0.0,0.0,1.437760e+15,3.117718e+06,1.437760e+15,1.437760e+15,audio-streaming,Non-Tor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373230,6.0,1.189162e+08,10434.109107,14532.582292,7.100962e+06,1.324509e+07,1460.000000,0.000000,773.016065,578.716198,...,0.0,0.0,0.0,0.0,1.436820e+15,3.264182e+07,1.436820e+15,1.436819e+15,chat,Tor
373231,6.0,1.171277e+08,7369.907904,4142.834225,7.756957e+06,2.604765e+06,1401.950374,0.000000,376.708801,317.820442,...,0.0,0.0,0.0,0.0,1.456330e+15,3.369741e+07,1.456330e+15,1.456330e+15,chat,Tor
373232,6.0,1.118895e+08,1648.654722,3052.762940,2.342838e+05,3.487852e+06,1422.761308,0.000000,195.313319,335.176988,...,0.0,0.0,0.0,0.0,1.438110e+15,1.978795e+07,1.438110e+15,1.438110e+15,chat,Tor
373233,6.0,9.386853e+07,248.859982,471.890747,2.925642e+04,5.455209e+05,926.141686,179.537499,401.194093,240.908037,...,0.0,0.0,0.0,0.0,1.436960e+15,3.650460e+07,1.436960e+15,1.436960e+15,chat,Tor
